In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [5]:
model_name = 'meta-llama/Meta-Llama-3-8B'
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model_name = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained('ZWG817/Llama3_Chat_Materials')
model.resize_token_embeddings(len(tokenizer))

model.load_adapter('ZWG817/Llama3_Chat_Materials')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/149k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

In [7]:
data = load_dataset("ZWG817/Abstract")
data_train = data["train"]
print(data_train)

#custom_data = load_dataset('json', data_files='data_eval.json')
#data_val = custom_data['train']

with open('materials.txt', 'r') as file:
    word_list = file.read().splitlines()

Generating train split:   0%|          | 0/13 [00:00<?, ? examples/s]

Dataset({
    features: ['publicationDate', 'title', 'abstract', 'id'],
    num_rows: 13
})


In [8]:
# data = load_dataset('csv', data_files="gdc.csv")
# data_train = concatenate_datasets([data_train, data["train"]])

In [9]:
new_data = {
    'title': word_list,
    'abstract': [s.replace('_', '') for s in word_list],
    'id': ['material'] * len(word_list),  # 假设新数据集中没有id信息
    'publicationDate': [None] * len(word_list)  # 假设新数据集中没有publicationDate信息
}
new_dataset = Dataset.from_dict(new_data)

data_train = concatenate_datasets([data_train, new_dataset])

new_data = {
    'title': [s.replace('_', '') for s in word_list],
    'abstract': word_list,
    'id': ['material'] * len(word_list),  # 假设新数据集中没有id信息
    'publicationDate': [None] * len(word_list)  # 假设新数据集中没有publicationDate信息
}
new_dataset = Dataset.from_dict(new_data)

data_train = concatenate_datasets([data_train, new_dataset])

In [10]:
data_val = data_train.select(range(200))

In [42]:
def generate_prompt(content):
    begin = "<s>[INST]"
    #syst = "<<SYS>> You are a helpful assistant, always answer as helpfully as possible.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "<<SYS>> In your role as a helpful scientific assistant, Read the following text. Does it give the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant. Make your answer include the material's molecular formula, and make your answer as short as possible.<</SYS>>\n"
    inst = content
    end = "[/INST]\n"
    prompt = (" ").join([begin, syst, inst, end])
    return prompt

print(generate_prompt(data_train[1]['abstract']))

<s>[INST] <<SYS>> You are a helpful assistant, Read the following text. Does it give the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant. Make your answer include the material's molecular formula, and make your answer as short as possible.<</SYS>>
 Magnetization dynamics in W/CoFeB, CoFeB/Pt and W/CoFeB/Pt multilayers was
investigated using spin-orbit-torque ferromagnetic resonance (SOT-FMR)
technique. An analytical model based on magnetization dynamics due to SOT was
used to fit heavy metal (HM) thickness dependence of symmetric and
antisymmetric components of the SOT-FMR signal. The analysis resulted in a
determination of the properties of HM layers, such as spin Hall angle and spin
diffusion length. The spin Hall angle of -0.36 and 0.09 has been found in the
W/CoFeB and CoFeB/Pt bilayers, respectively, which add up in the case of
W/CoFeB/Pt trilayer. More importantly, we have determined effective interfacial
spi

In [71]:
import numpy as np
import pandas as pd
df = pd.DataFrame()
df_tmp = pd.DataFrame()
df['molecular formula'] = []

In [ ]:
import re
results = []

for i,j in enumerate(data_train):
    print(i)
    input_prompt = generate_prompt(data_train[i]["abstract"])
    input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
    with torch.cuda.amp.autocast():
        generation_output = model.generate(
            input_ids=input_tokens,
            max_new_tokens=128,
            do_sample=True,
            top_k=5,
            top_p=0.9,
            temperature=0.2,
            repetition_penalty=1.1,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    #print(op)
    
    inst_index = op.find('[/INST]')
    
    if inst_index != -1:
        result = op[inst_index + len('[/INST]'):]
        # result = result.replace("<built-in function input>",'').replace("</built-in function input>",'').replace("</s>",'').replace("</s",'').replace("</",'').strip()
        if result != ' ':
            pattern = re.compile(r'<built-in function input> (.*?) </s>')

        # Use findall() to find all occurrences of the pattern
            extracted_elements = pattern.findall(result)
            print(extracted_elements)
            results.extend(extracted_elements)
    else:
        print("未找到'[/INST]'标记")
    if len(results) % 1000 == 0:
        df_tmp['molecular formula'] = results
        df = pd.concat([df,df_tmp])
        results = []
        df_tmp = pd.DataFrame()
        df.to_csv('molecular formula.csv',index = False)

0
[]
1
[]
2
[]
3
[]
4
['</s> 2-Co0.5Fe0.5']
5
[]
6
[]
7
[]
8
[]
9
[]
10
[]
11
['SrNi_2P_2']
12
[]
13
['Ca']
14
['CoO3']
15
['CoCO3']
16
['CuFe2O4']
17
['Cu3Fe4O']
18
['Fe2O3']
19
['Fe2MoO8']
20
['Fe3O4']
21
['Fe2As2']
22
['Fe3O4']
23
['Fe2Se2']
24
['Fe3O4']
25
['Fe4As4']
26
['Co_72Pt_28']
27
['GaMn_3N']
28
['NiNO_3']
29
['Mn2O4']
30
['Fe']
31
['Mn2O3']
32
['FeV2O4']
33
['V3Sb3']
34
['V3Sb5']
35
['GaMn_3N']
36
['NpCo_2']
37
['Fe3O4']
38
['GaMn_3N', 'Bi_5FeTi_3O_15']
39
['0.02']
40
['Fe3O4']
41
['Fe_65Co_35']
42
['Fe3O4']
43
['Fe_65Co_35']
44
['A2MMnO6']
45
['Mn3Sn']
46
['BaZn_xTi_yMn_zFe_12-x-y-zO_19']
47
['Fe2As2']
48
['Mn3+']
49
['Fe2Se2']
50
['CoFe2O4']
51
['FeAl_2-xCr_xO_4']
52
['NiNO_3']
53
['Fe3O4']
54
['NiNO_3']
55
['Cd_xFe_3-xO_4']
56
['NiNO_3']
57
['NiNO_3']
58
['NiNO_3']
59
['0.002']
60
['NiNO_3']
61
['0.02']
62
['0.03']
63
['NiNO_3']
64
['NiNO_3']
65
['Fe']
66
['MnO']
67
['NiNO_3']
68
['NiNO_3']
69
['Li_0.5Fe_2.5-xMn_xO_4']
70
['GaMn_3N']
71
['Fe_65Co_35']
72
['Fe_65Co_35']
7

In [22]:
input_tokens

tensor([[128000,  45147,  31868,  65562,     60,   1134,  39031,   2511,  73789,
            439,    264,  11190,  18328,    449,    264,   5357,    389,  15374,
             11,   3477,    279,  72758,   1495,    311,   8417,    422,    433,
          15407,    279,  46092,  84614,   6926,    369,    264,   3230,   3769,
             13,  12540,   1778,   2038,    387,   3118,     11,   1160,   1193,
            279,   3769,   6532,  16662,   1202,  20081,  46092,  84614,   6926,
          16134,    524,  39031,  40171,   5783,  22974,    532,    423,   4341,
            278,   9451,  89492,   3893,   5364,  13915,   7911,  16628,    320,
             72,  32989,      8,    304,    958,  10546,   9473,    198,  23045,
          50185,    320,   5484,     34,      8,  52170,     14,   7489,   6535,
             90,    508,  32816,   6251,   6535,     90,   1399,  32816,     33,
           6535,     90,    508,     92,   8693,     16,     13,    717,    198,
          20211,   5738,  99

In [23]:
model.save_pretrained('result', save_embedding_layers=True)

In [24]:
# for param in model.parameters():
#     print(param.dtype)

torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bflo

In [25]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(134933, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=4

In [26]:
# for name, param in model.named_parameters():
#     print(param.requires_grad, name, param.dtype)

In [27]:
# Verify which parameters are trainable
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"Trainable: {name}")

In [28]:
# for name, param in model.named_parameters():
#     print(f"Trainable: {name}", param.requires_grad)

In [29]:
# model.parameters()

In [30]:
# model.get_input_embeddings()

In [31]:
model.num_parameters()

In [32]:
# tokenizer.add_special_tokens({"pad_token": "<PAD>"})
# model.resize_token_embeddings(len(tokenizer))